# Bringing in the Data 

In [1]:
import random
import string
import sagemaker
import boto3
from sagemaker import get_execution_role
import s3fs
import pandas as pd
import datetime
import time
import numpy as np
import gc
from random import seed
from random import randint
import sklearn
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.dates as dates

In [7]:
import sys
# !{sys.executable} -m pip install PyAthena

import time
from pyathena import connect
import pandas as pd

start_time = time.time()

conn = connect(s3_staging_dir='s3://sec-edw-dev-processed/athena',
               region_name='us-east-1', work_group = 'sec-edw')

taq = pd.read_sql('SELECT * FROM "sec-edw-dev-finra"."bii_full_dukascopy_customers" limit 10000;', conn)

print("--- %s seconds ---" % (time.time() - start_time))

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
--- 10.349133253097534 seconds ---


In [22]:
taq.to_csv('taq_sample.csv', index=False) 

# Fuzzy Matching

In [8]:
pd.set_option('display.max_columns', None)
taq

,f,trdng_cntr,actv_trdng_cntr_cnt,symbol,date,order_rcpt_tm,order_type,order_size,order_side,order_price,nbb_price,nbb_size,nbo_price,nbo_size,tc_bid_offer,tc_size,iso_flag,rtl_inv_order_fl,rtbl_fl,ioc_fl,luld_impact,foreign_rte_fl,qt_leader_fl,wa_ex_price,wa_ex_tm,ex_shares,cncl_shares,rtd_shares,wa_rtd_ex,wa_rtd_ex_tm,spcl_hndlg_ind,action,orderid,mm_buy_ct,mm_sell_ct,share_prtcp_buy,share_prtcp_sell,trade_prtcp_buy,trade_prtcp_sell,cross_qt_shr_prtcp_buy,cross_qt_shr_prtcp_sell,cross_qt_trd_prtcp_buy,cross_qt_trd_prtcp_sell,mm_buy_cross_qt_ct,mm_sell_cross_qt_ct,date_formatted,price_decile,security_description,cusip,security_type,sip_symbol,old_symbol,listed_exchange,tape,nyse_industry_code,specialist_clearing_agent,specialist_clearing_number,specialist_post_number,specialist_panel,effective_date,customer_id,name,street_address,city,state,zipcode,phone_number,ssn,email
0,D,AT5625,7,LXRX,20190215,112527.341000,11,100,B,5.29,5.28,16,5.29,3,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,299862879,4.0,3.0,1188.8889,3100.0000,11.0,28.0,0.0,100.0000,0.0,1.0,0.0,1.0,2019-02-15,2,"Lexicon Pharmaceuticals, Inc.",528872302,A,LXRX,,Q,C,,,NaN,NaN,,NaN,187134,John Higgins,344 Michael Ways,Omer,Texas,32531.0,phone-number,159-37-5652,JHiggins@yahoo.com
1,D,AT3446,6,EIGI,20190219,132846.917000,11,500,B,7.10,7.09,12,7.10,2,,,N,N,N,Y,N,D,,NaN,NaN,0,500,0,,,Y,Order,651070191,2.0,4.0,1300.0000,1407.0000,13.0,15.0,100.0,100.0000,1.0,1.0,1.0,1.0,2019-02-19,2,"Endurance International Group Holdings, Inc.",29272B105,A,EIGI,,Q,C,,,NaN,NaN,,NaN,366020,Jennifer Mccarty,66269 Hill Rue,Blomkest,Pennsylvania,703.0,phone-number,382-88-0214,Jennifer_Mccarty@aol.com
2,D,AT3446,6,CBPO,20190226,105229.167828,11,100,B,81.63,81.48,4,81.63,4,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Canceled,78993292,2.0,5.0,400.0000,3550.0000,4.0,11.0,0.0,0.0000,0.0,0.0,0.0,0.0,2019-02-26,10,"China Biologic Products Holdings, Inc.",G21515104,A,CBPO,,Q,C,,,NaN,NaN,,NaN,837174,Christian Oconnor,646 Graham Camp,Blackshear,California,33693.0,phone-number,324-76-2195,Christian.O@hotmail.com
3,D,AT4917,7,TECD,20190320,150727.538446,11,22,SS,104.79,104.79,1,104.89,1,,,N,N,N,Y,N,D,,NaN,NaN,0,22,0,,,Y,Canceled,90419151,7.0,8.0,4797.0000,8669.0000,81.0,102.0,0.0,100.0000,0.0,1.0,0.0,1.0,2019-03-20,10,Tech Data Corp,878237106,A,TECD,,Q,C,,,NaN,NaN,,NaN,221875,David Taylor,770 Young Inlet Apt. 676,Keenan,Idaho,49377.0,phone-number,049-42-4677,DavidTaylor@verizon.com
4,D,AT3446,6,CBPO,20190226,131637.345000,11,100,S,80.98,80.98,1,81.10,2,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,253866574,2.0,5.0,400.0000,3550.0000,4.0,11.0,0.0,0.0000,0.0,0.0,0.0,0.0,2019-02-26,10,"China Biologic Products Holdings, Inc.",G21515104,A,CBPO,,Q,C,,,NaN,NaN,,NaN,86436,Derek Acosta,8298 Huynh Roads Suite 141,Harwich Port,Oklahoma,73506.0,phone-number,511-05-0040,Derek_Acosta91@xfinity.com
5,D,TC3088,5,ATRO,20190327,100616.342433,11,100,S,30.53,30.53,2,30.66,3,,,N,N,Y,Y,N,D,,NaN,NaN,0,100,0,,,N,Canceled,89008382,0.0,1.0,0.0000,193.0000,0.0,4.0,0.0,0.0000,0.0,0.0,0.0,0.0,2019-03-27,7,Astronics Corp,046433108,A,ATRO,,Q,C,,,NaN,NaN,,NaN,990535,Eric Hooper,1759 Vincent Locks,Continental,Arizona,82945.0,phone-number,286-59-9079,Eric.Hooper31@verizon.com
6,D,AT4917,7,ORN,20190219,155424.397000,11,100,B,4.45,4.43,7,4.45,5,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,289148688,3.0,2.0,2224.5721,4062.0000,30.0,31.0,700.0,2900.0000,7.0,24.0,1.0,1.0,2019-02-19,2,"Orion Group Holdings, Inc.",68628V308,A,ORN,,N,A,126E,G,5.0,8.0,H,20090514.0,731165,Erik Gibbs,748 Vasquez Drive Suite 580,Alcoa Center,Arkansas,7578.0,phone-number,470-76-1159,Erik_Gibbs@yahoo.com
7,D,AT4917,7,ORN,20190214,111701.988800,11,100,B,4.04,4.03,1,4.04,1,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Canceled,344811343,1.0,2.0,10.0000,697.0000,1.0,6.0,0.0,0.0000,0.0,0.0,0.0,0.0,2019-02-14,2,"Orion Group Holdings, Inc.",68628V308,A,ORN,,N,A,126E,G,5.0,8.0,H,20090514.0,340864,Adrienne Tran,82184 Samantha Club,Fayette,New Mexico,6975.0,phone-number,264-91-3032,Adrienne.Tran85@aol.com
8,

In [10]:
quotes = taq[taq['action'] == 'Order']
pd.set_option('display.max_columns', None)
quotes

,f,trdng_cntr,actv_trdng_cntr_cnt,symbol,date,order_rcpt_tm,order_type,order_size,order_side,order_price,nbb_price,nbb_size,nbo_price,nbo_size,tc_bid_offer,tc_size,iso_flag,rtl_inv_order_fl,rtbl_fl,ioc_fl,luld_impact,foreign_rte_fl,qt_leader_fl,wa_ex_price,wa_ex_tm,ex_shares,cncl_shares,rtd_shares,wa_rtd_ex,wa_rtd_ex_tm,spcl_hndlg_ind,action,orderid,mm_buy_ct,mm_sell_ct,share_prtcp_buy,share_prtcp_sell,trade_prtcp_buy,trade_prtcp_sell,cross_qt_shr_prtcp_buy,cross_qt_shr_prtcp_sell,cross_qt_trd_prtcp_buy,cross_qt_trd_prtcp_sell,mm_buy_cross_qt_ct,mm_sell_cross_qt_ct,date_formatted,price_decile,security_description,cusip,security_type,sip_symbol,old_symbol,listed_exchange,tape,nyse_industry_code,specialist_clearing_agent,specialist_clearing_number,specialist_post_number,specialist_panel,effective_date,customer_id,name,street_address,city,state,zipcode,phone_number,ssn,email
0,D,AT5625,7,LXRX,20190215,112527.341,11,100,B,5.29,5.28,16,5.29,3,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,299862879,4.0,3.0,1188.8889,3100.0000,11.0,28.0,0.0000,100.0000,0.0,1.0,0.0,1.0,2019-02-15,2,"Lexicon Pharmaceuticals, Inc.",528872302,A,LXRX,,Q,C,,,NaN,NaN,,NaN,187134,John Higgins,344 Michael Ways,Omer,Texas,32531.0,phone-number,159-37-5652,JHiggins@yahoo.com
1,D,AT3446,6,EIGI,20190219,132846.917,11,500,B,7.10,7.09,12,7.10,2,,,N,N,N,Y,N,D,,NaN,NaN,0,500,0,,,Y,Order,651070191,2.0,4.0,1300.0000,1407.0000,13.0,15.0,100.0000,100.0000,1.0,1.0,1.0,1.0,2019-02-19,2,"Endurance International Group Holdings, Inc.",29272B105,A,EIGI,,Q,C,,,NaN,NaN,,NaN,366020,Jennifer Mccarty,66269 Hill Rue,Blomkest,Pennsylvania,703.0,phone-number,382-88-0214,Jennifer_Mccarty@aol.com
4,D,AT3446,6,CBPO,20190226,131637.345,11,100,S,80.98,80.98,1,81.10,2,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,253866574,2.0,5.0,400.0000,3550.0000,4.0,11.0,0.0000,0.0000,0.0,0.0,0.0,0.0,2019-02-26,10,"China Biologic Products Holdings, Inc.",G21515104,A,CBPO,,Q,C,,,NaN,NaN,,NaN,86436,Derek Acosta,8298 Huynh Roads Suite 141,Harwich Port,Oklahoma,73506.0,phone-number,511-05-0040,Derek_Acosta91@xfinity.com
6,D,AT4917,7,ORN,20190219,155424.397,11,100,B,4.45,4.43,7,4.45,5,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,289148688,3.0,2.0,2224.5721,4062.0000,30.0,31.0,700.0000,2900.0000,7.0,24.0,1.0,1.0,2019-02-19,2,"Orion Group Holdings, Inc.",68628V308,A,ORN,,N,A,126E,G,5.0,8.0,H,20090514.0,731165,Erik Gibbs,748 Vasquez Drive Suite 580,Alcoa Center,Arkansas,7578.0,phone-number,470-76-1159,Erik_Gibbs@yahoo.com
11,D,AT4917,7,ORN,20190326,135430.436,11,1000,B,2.98,2.96,2,2.98,2,,,N,N,N,Y,N,D,,NaN,NaN,0,1000,0,,,Y,Order,546928876,5.0,4.0,5435.0000,2672.0000,60.0,35.0,2300.0000,240.0000,23.0,3.0,3.0,2.0,2019-03-26,1,"Orion Group Holdings, Inc.",68628V308,A,ORN,,N,A,126E,G,5.0,8.0,H,20090514.0,876832,Monica Alvarez,21737 Vincent Inlet Suite 455,Pickerington,South Dakota,74994.0,phone-number,501-82-6197,MonicaAlvarez@att.com
12,D,AT4917,7,ORN,20190201,122313.259,11,100,B,4.14,4.12,2,4.14,1,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,464572157,3.0,3.0,701.0000,656.0000,7.0,15.0,100.0000,0.0000,1.0,0.0,1.0,0.0,2019-02-01,2,"Orion Group Holdings, Inc.",68628V308,A,ORN,,N,A,126E,G,5.0,8.0,H,20090514.0,877086,Lindsey Chavez,60150 Mason Parkway Suite 271,Keams Canyon,Arkansas,99476.0,phone-number,188-06-5300,Lindsey_Chavez@gmail.com
13,D,AT4917,7,ERA,20190326,101118.051,11,100,B,11.84,11.65,1,11.84,3,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,287068444,3.0,2.0,300.0000,208.0000,4.0,3.0,91.0000,100.0000,1.0,1.0,1.0,1.0,2019-03-26,4,Era Group Inc.,26885G109,A,ERA,,N,A,126E,G,5.0,8.0,H,20130122.0,595246,Melissa Kim,55311 Salas Extensions,Libertyville,Vermont,11728.0,phone-number,359-09-8811,Melissa_K@yahoo.com
14,D,AT4917,7,ERA,20190326,114605.502,11,100,B,11.67,11.60,1,11.67,1,,,N,N,N,Y,N,D,,NaN,NaN,0,100,0,,,Y,Order,586281526,3.0,2.0,300.0000,208.0000,4.0,3.0,91.0000,100.0000,1.0,1.0,1.0,1.0,2019-03-26,4,Era Group Inc.,26885G109,A,ERA,,N,A,126E,G,5.0,8.0,H,20130122.0,208557,Brian Webb,573 Murphy Knoll Apt. 660,Hercules,Michigan,847

In [11]:
trades = taq[taq['action'] == 'Executed']
pd.set_option('display.max_columns', None)
trades

,f,trdng_cntr,actv_trdng_cntr_cnt,symbol,date,order_rcpt_tm,order_type,order_size,order_side,order_price,nbb_price,nbb_size,nbo_price,nbo_size,tc_bid_offer,tc_size,iso_flag,rtl_inv_order_fl,rtbl_fl,ioc_fl,luld_impact,foreign_rte_fl,qt_leader_fl,wa_ex_price,wa_ex_tm,ex_shares,cncl_shares,rtd_shares,wa_rtd_ex,wa_rtd_ex_tm,spcl_hndlg_ind,action,orderid,mm_buy_ct,mm_sell_ct,share_prtcp_buy,share_prtcp_sell,trade_prtcp_buy,trade_prtcp_sell,cross_qt_shr_prtcp_buy,cross_qt_shr_prtcp_sell,cross_qt_trd_prtcp_buy,cross_qt_trd_prtcp_sell,mm_buy_cross_qt_ct,mm_sell_cross_qt_ct,date_formatted,price_decile,security_description,cusip,security_type,sip_symbol,old_symbol,listed_exchange,tape,nyse_industry_code,specialist_clearing_agent,specialist_clearing_number,specialist_post_number,specialist_panel,effective_date,customer_id,name,street_address,city,state,zipcode,phone_number,ssn,email
31,D,TC7687,4,FARM,20190313,155914.795,11,8,SS,23.9300,23.9300,2,23.9900,6,,,N,N,N,Y,N,D,,23.9300,0.000,8,0,0,,,N,Executed,542255470,1.0,1.0,124.0000,1248.0000,4.0,7.0,0.0000,0.0000,0.0,0.0,0.0,0.0,2019-03-13,7,Farmer Bros Co,307675108,A,FARM,,Q,C,,,NaN,NaN,,NaN,136344,Nancy Gomez,785 Willis Course,Fort Gaines,Idaho,6619.0,phone-number,300-80-2760,Nancy_G45@hotmail.com
42,D,TC1459,7,LPCN,20190213,100136.226,11,30,B,1.8300,1.7200,3,1.7500,49,,,N,N,Y,N,N,D,,1.7393,0.022,30,0,0,,,N,Executed,543312060,1.0,1.0,11638.0000,8528.0000,11.0,15.0,0.0000,0.0000,0.0,0.0,0.0,0.0,2019-02-13,1,Lipocine Inc.,53630X104,A,LPCN,,Q,C,,,NaN,NaN,,NaN,238042,Charles Lopez,190 Johnson Club,Cowdrey,Alaska,55877.0,phone-number,498-99-3036,Lopez_Charles88@aol.com
69,D,TC7687,4,CTRL,20190212,131350.180,10,300,SS,NaN,18.2000,2,18.2200,1,,,N,N,N,Y,Y,D,,18.2100,0.001,300,0,0,,,Y,Executed,326513982,1.0,3.0,95298.0000,12126.0000,305.0,65.0,37424.0000,5060.0000,58.0,20.0,1.0,3.0,2019-02-12,None,Control4 Corporation,21240D107,A,CTRL,,Q,C,,,NaN,NaN,,NaN,457492,Matthew Collier,4413 Smith Rapid,Zelienople,New Jersey,20987.0,phone-number,259-62-7172,Matthew.C16@xfinity.com
89,D,AT3446,6,LPCN,20190312,95248.047,11,300,S,2.3000,2.3000,91,2.3100,26,,,N,N,N,Y,N,D,,2.3015,0.000,300,0,0,,,N,Executed,378544161,7.0,4.0,62508.9778,16597.0000,342.0,88.0,13219.4728,500.0000,80.0,5.0,4.0,1.0,2019-03-12,1,Lipocine Inc.,53630X104,A,LPCN,,Q,C,,,NaN,NaN,,NaN,878001,Nicole Mendoza,420 Brown Street Suite 153,Fort Myers Villas,New Hampshire,28491.0,phone-number,398-49-4742,Nicole.Mendoza@comcast.net
111,D,AT3446,6,TVTY,20190213,111613.151,11,600,SS,20.9500,20.9500,4,20.9700,4,,,N,N,N,N,N,D,,20.9500,0.001,600,0,0,,,Y,Executed,425160815,3.0,5.0,3100.0000,600.0000,13.0,6.0,100.0000,200.0000,1.0,2.0,1.0,2.0,2019-02-13,6,"Tivity Health, Inc. Common Stock",88870R102,A,TVTY,,Q,C,,,NaN,NaN,,NaN,349567,Amy Baker,29741 Jacobson Courts,Glendive,Kansas,60071.0,phone-number,227-63-2516,Amy_B@xfinity.com
120,D,TC7687,4,TECD,20190214,125948.317,10,4,B,NaN,103.1500,1,103.2300,1,,,N,N,Y,N,Y,D,,103.1984,0.002,4,0,0,,,N,Executed,175501457,1.0,1.0,3774.0000,16412.0000,50.0,172.0,309.0000,1379.0000,4.0,12.0,1.0,1.0,2019-02-14,None,Tech Data Corp,878237106,A,TECD,,Q,C,,,NaN,NaN,,NaN,536480,Wesley Cox,00730 Yang Summit Suite 254,Shipshewana,Florida,53351.0,phone-number,833-99-6918,WesleyCox@hotmail.com
158,D,TC7687,4,INGN,20190311,112145.572,10,25,B,NaN,94.7900,1,94.8700,2,,,N,N,Y,N,Y,D,,94.8436,0.003,25,0,0,,,N,Executed,464751652,2.0,3.0,28884.0000,12718.0000,273.0,131.0,7161.0000,1750.0000,39.0,11.0,2.0,2.0,2019-03-11,None,Inogen Inc,45780L104,A,INGN,,Q,C,,,NaN,NaN,,NaN,342858,Allison Martinez,4279 Walter Station,Valmont,West Virginia,34216.0,phone-number,506-52-2935,Martinez_Allison41@xfinity.com
163,D,TC7687,4,HCLP,20190329,154547.361,11,100,B,4.4700,4.4600,5,4.4700,8,,,N,N,N,Y,N,D,,4.4700,0.000,100,0,0,,,N,Executed,326586983,2.0,2.0,41147.0000,51288.0000,95.0,225.0,12553.0000,11290.0000,32.0,25.0,2.0,2.0,2019-03-29,2,Hi-Crush Partners LP,428337109,A,HCLP,,N,A,304C,CDRG,8430.0,5.0,D,20120816.0,588217,Michelle Gomez,227 Shelby Vista Sui

In [ ]:
import pandas as pd
import numpy as np

quotes = quotes.to_numpy()

trades = trades.to_numpy()

# So we want to match a and b in the end 
#call searchsorted() to find index in t1 for every value in t2:
#Find the indices into a sorted Series self such that, if the corresponding elements in value were inserted 
#before the indices, the order of self would be preserved.
idx = np.searchsorted(quotes, trades) - 1
mask = idx >= 0

df = pd.DataFrame({"quotes":quotes[idx][mask], "trades":trades[mask]})
df.head()

In [ ]:
# To view this result by graph:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pylab as pl
pl.figure(figsize=(18, 4))
pl.vlines(pd.Series(t1), 0, 1, colors="g", lw=1)
pl.vlines(df.t1, 0.3, 0.7, colors="r", lw=2)
pl.vlines(df.t2, 0.3, 0.7, colors="b", lw=2)
pl.margins(0.02)